In [51]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [52]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
toronto_neighborhoods = pd.DataFrame(columns=column_names)
toronto_neighborhoods

,PostalCode,Borough,Neighborhood


### Fetch Wikipedia page details and use BeautifulSoup library to fetch Wikitable class details

In [53]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

### Loop through each wikitable row and fetch individual columns values to build the dataframe values using conditions specified in assignment 

In [54]:
toronto_neighborhoods.drop(toronto_neighborhoods.index, inplace=True)
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    # Assign each column/cell of a given row to a variable 
    if len(cells) == 3:
        pc = cells[0].find(text=True)
        bo = cells[1].find(text=True)
        nb = cells[2].find(text=True)
        
        # If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
        # if Neighborhood contains value of 'Not assigned', Assign value in Borough to Neighborhood 
        if nb.find('Not assigned',0,13) != -1:
            nb = bo
        
        # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
        if bo != 'Not assigned':           
            # More than one neighborhood can exist in one postal code area and those rows has to be combined into one row with the neighborhoods separated with a comma.
                toronto_neighborhoods = toronto_neighborhoods.append({'PostalCode': pc, 'Borough': bo, 'Neighborhood': nb}, ignore_index=True)
toronto_neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [8]:
toronto_neighborhoods.shape

(212, 3)

In [82]:
toronto_neighborhoods.drop(toronto_neighborhoods.index, inplace=True)
num = 0 
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    # Assign each column/cell of a given row to a variable 
    if len(cells) == 3:
        pc = cells[0].find(text=True)
        bo = cells[1].find(text=True)
        nb = cells[2].find(text=True)
        # Replace new line character with blank
        nb = nb.replace("\n","")
        
        # If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
        # if Neighborhood contains value of 'Not assigned', Assign value in Borough to Neighborhood 
        if nb.find('Not assigned',0,13) != -1:
            nb = bo
        
        # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
        if bo != 'Not assigned':     
            #print( len(toronto_neighborhoods.Postal_code))
            # More than one neighborhood can exist in one postal code area and those rows has to be combined into one row with the neighborhoods separated with a comma.
            if num == 0:
                # First Successful Entry
                toronto_neighborhoods = toronto_neighborhoods.append({'PostalCode': pc, 'Borough': bo, 'Neighborhood': nb}, ignore_index=True)
                num += 1
            else:
                # If the Postal Code already exists in toronto neighborhood, add value to existing Neighborhood of Postal Code
                found = 0
                for index, row in toronto_neighborhoods.iterrows() :
                    if row['PostalCode'] == pc:
                        nb = row['Neighborhood'] + ',' + nb
                        found = 1
                        row['Neighborhood'] = nb
                # If the Postal Code is not found, add new value
                if found == 0:    
                    toronto_neighborhoods = toronto_neighborhoods.append({'PostalCode': pc, 'Borough': bo, 'Neighborhood': nb}, ignore_index=True)
        
toronto_neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [85]:
# To view full large value for postal code 
print('Postal Code:', toronto_neighborhoods.iloc[87][0], ' Neighborhood (Full large Concatenated Value):' , toronto_neighborhoods.iloc[87][2])

Postal Code: M5V  Neighborhood (Full large Concatenated Value): CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
